In [16]:
import unittest
from pydantic import BaseModel, Field, field_validator, model_validator
import plotly.graph_objects as go
import numpy as np
from typing import List, Optional
import uuid

uses a hexagonal coordinate system

ie, going clockwise around origin, pointy side up

| q | r | s |
 |----|--|--|
1 |-1 |0
1 |0 |-1
0 |1 |-1
-1| 1 |0
-1 |0 |1
0 |-1| 1

q is in the up right orientation  
r is rows in the down orientation  
s is in the up left direction  

two coordinates that are adjacant

https://www.redblobgames.com/grids/hexagons/

In [ ]:
class HexCoordinate(BaseModel):
    q: int
    r: int
    s: int

    @field_validator('s', mode='before')
    @classmethod
    def validate_cube_coordinates(cls, v, values):
        if values.data['q'] + values.data['r'] + v != 0:
            raise ValueError('Invalid cube coordinates')
        return v

class GamePiece(BaseModel):
    hex_coordinates: list[HexCoordinate]
    icon: str = "�"
    team: str
    piece_id: int = Field(default_factory=lambda: str(uuid.uuid4()))
    location: str = Field(default='offboard') # 'offboard', 'board', 'stacked'

    @field_validator('team')
    @classmethod
    def validate_team(cls, v):
        if v not in ['black', 'white']:
            raise ValueError('Team must be either "black" or "white"')
        return v

class Spider(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates or [], 
            team=team,
            icon="🕷️",
            location='offboard'
            )

class Ant(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates or [], 
            team=team,
            icon="🐜",
            location='offboard'
            )

class Beetle(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🪲",
            location='offboard'
            )

class Grasshopper(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🦗",
            location='offboard'
            )

class QueenBee(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates or [], 
            team=team,
            icon="🐝",
            location='offboard'
            )

class Ladybug(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🐞",
            location='offboard'
            )

class Mosquito(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🦟",
            location='offboard'
            )

class BoardState(BaseModel):
    pieces: dict = Field(default_factory=dict)

    def add_piece(self, piece_id: str, piece, coordinates: HexCoordinate):
        if piece_id in self.pieces:
            raise ValueError('Piece ID already exists on the board')
        self.pieces[piece_id] = piece
        self.pieces[piece_id].hex_coordinates = coordinates
        self.pieces[piece_id].location = 'board'

class Player(BaseModel):
    name: str
    team: str
    pieces: List[GamePiece] = Field(default_factory=None)

    def __init__(self, name: str, team: str, pieces: Optional[List[GamePiece]] = None):
        if pieces is None:
            pieces = [
                *[Ant(team=team) for _ in range(12)],
                QueenBee(team=team)
                # super basic mode to start. add more pieces later
            ]
        super().__init__(name=name, team=team, pieces=pieces)

    @field_validator('team')
    @classmethod
    def validate_team(cls, v):
        if v not in ['black', 'white']:
            raise ValueError('Team must be either "black" or "white"')
        return v

class GameState(BaseModel):
    turn: int = Field(0, ge=0)
    white_player: Player = Player(name='white', team='white', pieces=[])
    black_player: Player = Player(name='black', team='black', pieces=[])
    current_team: str = Field(default='white')
    board_state: BoardState = Field(default_factory=BoardState)
    verbose: bool = Field(default=True)

    def __init__(self, white_player: Player = None, black_player: Player = None, **data):
        if white_player is None:
            white_player = Player(name='white', team='white')
        if black_player is None:
            black_player = Player(name='black', team='black')
        super().__init__(white_player=white_player, black_player=black_player, **data)

    def get_available_spaces(self):
        if len(self.board_state.pieces) == 0:
            if self.verbose:
                print("No pieces on the board, returning center hex (0,0,0) as available space.")
            return HexCoordinate(q=0, r=0, s=0)  # If no pieces are on the board, return the center hex

        occupied = { (coord.q, coord.r, coord.s) for piece in self.board_state.pieces.values() for coord in piece.hex_coordinates }
        adjacent = set()
        for q, r, s in occupied:
            for dq, dr, ds in [(1, -1, 0), (1, 0, -1), (0, 1, -1), (-1, 1, 0), (-1, 0, 1), (0, -1, 1)]:
                adjacent.add((q + dq, r + dr, s + ds))
        relative = adjacent - occupied
        coords = []
        for coord in relative:
            coords.append(HexCoordinate(q=coord[0], r=coord[1], s=coord[2]))
        return coords

    @classmethod
    def create(cls, white_player: Player, black_player: Player):
        white = Player(default_factory=lambda: Player(name='white', team='white', pieces=white_player.pieces))
        black = Player(default_factory=lambda: Player(name='black', team='black', pieces=black_player.pieces))
        return cls(
            turn=1,
            white_player=white,
            black_player=black,
            current_team='white',
            board_state=BoardState()
        )

    @model_validator(mode='after')
    def validate_current_team(self):
        if self.current_team not in ['white', 'black']:
            raise ValueError('Current team must be either "white" or "black"')
        return self

class Turn(BaseModel):
    player: str
    piece_id: str
    action_type: str = Field(default_factory=lambda: ['move', 'place', 'forfeit'])
    target_coordinates: Optional[HexCoordinate] = None




In [18]:
game = GameState()
game.black_player

Player(name='black', team='black', pieces=[Ant(hex_coordinates=[], icon='🐜', team='black', piece_id='814ed9c4-33c7-4abc-bec5-a19a77c55075', location='offboard'), Ant(hex_coordinates=[], icon='🐜', team='black', piece_id='6bd6729b-5532-4bc6-b28f-0e26d4224f83', location='offboard'), Ant(hex_coordinates=[], icon='🐜', team='black', piece_id='abff61a3-b0bc-43b9-99e7-90c54c8c5558', location='offboard'), Ant(hex_coordinates=[], icon='🐜', team='black', piece_id='723cba2e-5fb8-4b44-8b46-7a925547cdc8', location='offboard'), Ant(hex_coordinates=[], icon='🐜', team='black', piece_id='7e053757-9a47-45a6-a2eb-3f2cf9fe1599', location='offboard'), Ant(hex_coordinates=[], icon='🐜', team='black', piece_id='94783bd4-1581-4202-92fa-359defb86208', location='offboard'), Ant(hex_coordinates=[], icon='🐜', team='black', piece_id='91f55131-ddf4-46d1-a40b-045087c6f71f', location='offboard'), Ant(hex_coordinates=[], icon='🐜', team='black', piece_id='a3a03e22-a116-4f3b-a3ca-4144fa453fc5', location='offboard'), Ant(h

In [19]:
piece1 = game.black_player.pieces[0]
piece1.hex_coordinates = [HexCoordinate(q=1, r=-1, s=0)]
piece2 = game.white_player.pieces[0]
piece2.hex_coordinates = [HexCoordinate(q=2, r=-2, s=0)]


In [20]:
game.get_available_spaces()

# add piece1 and piece2 to the board state for visualization
game.board_state.add_piece(piece1.piece_id, piece1)
game.board_state.add_piece(piece2.piece_id, piece2)

game.get_available_spaces()


No pieces on the board, returning center hex (0,0,0) as available space.


[HexCoordinate(q=3, r=-3, s=0),
 HexCoordinate(q=0, r=-1, s=1),
 HexCoordinate(q=1, r=0, s=-1),
 HexCoordinate(q=0, r=0, s=0),
 HexCoordinate(q=1, r=-2, s=1),
 HexCoordinate(q=2, r=-1, s=-1),
 HexCoordinate(q=2, r=-3, s=1),
 HexCoordinate(q=3, r=-2, s=-1)]

adjacant

In [21]:
def are_adjacent(coord1: GamePiece, coord2: GamePiece, verbose: bool) -> bool:
    dq = abs(coord1.q - coord2.q)
    dr = abs(coord1.r - coord2.r)
    ds = abs(coord1.s - coord2.s)
    distance = (dq + dr + ds) == 2
    if verbose:
        if distance:
            print("The coordinates are adjacent")
        else:
            print(f"The coordinates are not adjacent, distance is {dq + dr + ds}")

    return distance

are_adjacent(piece1.hex_coordinates[0], piece2.hex_coordinates[0], True)

The coordinates are adjacent


True

In [22]:
def hex_to_pixel(coord: HexCoordinate, size: float = 1.0):
    """Convert hex coordinate to pixel position for plotting."""
    x = size * (3/2 * coord.q)
    y = size * (np.sqrt(3)/2 * coord.q + np.sqrt(3) * coord.r)
    return x, y

def get_hexagon_vertices(x: float, y: float, size: float = 1.0):
    """Get vertices of a hexagon centered at (x, y)."""
    angles = np.linspace(0, 2*np.pi, 7)  # 7 points to close the hexagon
    vertices_x = x + size * np.cos(angles)
    vertices_y = y + size * np.sin(angles)
    return vertices_x, vertices_y



In [ ]:
def visualize_game_board(pieces: list[GamePiece], show_empty_hexes: Optional[list[HexCoordinate]] = None):
    """
    Visualize game pieces on hex coordinates
    """
    fig = go.Figure()
    
    # CUSTOMIZATION: Adjust hexagon size here
    hex_size = 0.95
    
    # CUSTOMIZATION: Team colors mapping
    team_colors = {
        "black": "#1D1A1A",  # Light gray for black team
        "white": "#FFFFFF",  # White for white team
        # Add more team colors as needed
    }
    
    team_border_colors = {
        "black": "#000000",  # Black border
        "white": "#808080",  # Gray border for visibility
    }
    
    # Draw empty hexes if provided
    if show_empty_hexes:
        for coord in show_empty_hexes:
            center_x, center_y = hex_to_pixel(coord, size=1.0)
            hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
            
            fig.add_trace(go.Scatter(
                x=hex_x,
                y=hex_y,
                fill='toself',
                fillcolor='#F5F5F5',
                line=dict(color='lightgray', width=1),
                mode='lines',
                showlegend=False,
                hoverinfo='text',
                hovertext=f'Empty: q={coord.q}, r={coord.r}, s={coord.s}',
            ))
    
    # Draw hexes with game pieces
    for piece in pieces:
        # Each piece can occupy multiple hexes
        for coord in piece.hex_coordinates:
            center_x, center_y = hex_to_pixel(coord, size=1.0)
            hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
            
            # Get team colors
            fill_color = team_colors.get(piece.team, 'lightgray')
            line_color = team_border_colors.get(piece.team, 'gray')
            
            # Draw the hexagon
            fig.add_trace(go.Scatter(
                x=hex_x,
                y=hex_y,
                fill='toself',
                fillcolor=fill_color,
                line=dict(color=line_color, width=2),
                mode='lines',
                showlegend=False,
                hoverinfo='text',
                hovertext=f'({piece.team})<br>Position: ({coord.q},{coord.r},{coord.s})',
            ))
            
            # Add piece icon in the center
            fig.add_trace(go.Scatter(
                x=[center_x],
                y=[center_y],
                mode='text',
                text=[piece.icon],
                textfont=dict(size=60, color='black'),  # CUSTOMIZATION: Icon size
                showlegend=False,
                hoverinfo='skip'
            ))
            

            fig.add_trace(go.Scatter(
                x=[center_x],
                y=[center_y - 0.3],
                mode='text',
                text=[f'({coord.q},{coord.r},{coord.s})'],
                textfont=dict(size=8, color='gray'),
                showlegend=False,
                hoverinfo='skip'
            ))
    
    # CUSTOMIZATION: Adjust plot appearance here
    fig.update_layout(
        title='Current State of the Game Board',
        xaxis=dict(
            scaleanchor='y',
            scaleratio=1,
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        plot_bgcolor='white',
        width=800,
        height=800,
    )
    
    fig.show()

visualize_game_board(game.board_state.pieces, show_empty_hexes=game.get_available_spaces())